<a href="https://colab.research.google.com/github/sayanbanerjee32/learning-equality-curriculum-recommendations/blob/main/lecr_SentTrans_topic_emmbd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle
## install collab specific libraries
if is_colab:
    !pip install -Uq datasets
    !pip install -Uq transformers
    !pip install -Uq sentencepiece 
    !pip install -Uq evaluate
    !pip install -Uq pynvml
!pip install -Uq sentence_transformers
!pip install -Uq faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.2 MB/s eta 0:0

In [3]:
# # This get the RAPIDS-Colab install files and test check your GPU.  Run cells 1 and 2 only.
# # Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/env-check.py

In [4]:
# # This will update the Colab environment and restart the kernel.
# !bash rapidsai-csp-utils/colab/update_gcc.sh
# import os
# os._exit(00)

In [5]:
# ## Installing CondaColab.  This will restart your kernel again
# import condacolab
# condacolab.install()

In [6]:
# # you can now run the rest of the cells as normal
# import condacolab
# condacolab.check()

In [7]:
# # Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# # The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
# # The <packages> option are default blank or 'core'.  By default, we install RAPIDSAI and BlazingSQL.  The 'core' option will install only RAPIDSAI and not include BlazingSQL, 
# !python rapidsai-csp-utils/colab/install_rapids.py nightly
# import os
# os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
# os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
# os.environ['CONDA_PREFIX'] = '/usr/local'

In [8]:
from fastkaggle import *
from pathlib import Path
import pandas as pd
import numpy as np # linear algebra
from tqdm.auto import tqdm
import gc

import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from datasets import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


In [9]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
comp = 'learning-equality-curriculum-recommendations'
if is_colab:
    model_save_path = Path('/content/'+comp+'_out/models')
    data_save_path = Path('/content/drive/MyDrive/Colab Notebooks/data/'+comp)
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')
    data_save_path = Path('/kaggle/working/'+comp)

In [10]:
# import colab libraries
if is_colab:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [12]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [13]:
path = setup_comp(comp)
path

100%|██████████| 254M/254M [00:11<00:00, 22.8MB/s]


Path('learning-equality-curriculum-recommendations')

### read all input files

In [14]:
# load the data into pandas dataframes
topics_df = pd.read_csv(path / "topics.csv", index_col=0).fillna({"title": "", "description": ""})
correlations_df = pd.read_csv(path / "correlations.csv", index_col=0)

In [15]:
print(topics_df.shape, correlations_df.shape)
correlations_df.head()

(76972, 8) (61517, 1)


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4


In [16]:
### sample records for colab only - test bed
if is_colab:
    topic_ids = correlations_df.reset_index()['topic_id'].tolist()
    topics_df = topics_df.query(f'id in {topic_ids}')
correlations_df.shape, topics_df.shape

((61517, 1), (61517, 8))

#### generate topic embedding

In [17]:
# model_nm = 'microsoft/mdeberta-v3-base'
# MAX_LEN = 256
model_nm = 'all-MiniLM-L6-v2'
MAX_LEN = 256
batch_size = 64
SELECT_TOP_N = 25

In [18]:
# Load models
# model = AutoModel.from_pretrained(model_nm)
# model.eval()
# model.to(device)

# tokenizer = AutoTokenizer.from_pretrained(model_nm)
## sentence transformer
model = SentenceTransformer(model_nm)

In [19]:
# helper function cleaning GPU memory
def report_gpu():
    if is_colab: print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [20]:
# create data set
topic_dataset = Dataset.from_pandas(topics_df)
topic_dataset

Dataset({
    features: ['title', 'description', 'channel', 'category', 'level', 'language', 'parent', 'has_content', 'id'],
    num_rows: 61517
})

In [21]:
# concatenate title and description
def concatenate_topic_text(examples):
    return {
        "concat_text": (examples["title"]
        + " \n "
        + examples["description"].replace('\n',' ').strip()
        ).strip()
    }
topic_dataset = topic_dataset.map(concatenate_topic_text)

Map:   0%|          | 0/61517 [00:00<?, ? examples/s]

In [22]:
# test embedding
report_gpu()
# embedding = get_embeddings(topic_dataset["concat_text"][0])
embedding = model.encode(topic_dataset["concat_text"][0])
embedding.shape

GPU:0
no processes are running


(384,)

In [23]:
report_gpu()
topic_embeddings_dataset = topic_dataset.map(
    # lambda x: {"embeddings": get_embeddings(x["concat_text"]).detach().cpu().numpy()[0]}
    lambda x: {"embeddings": model.encode(x["concat_text"])},
    batched=True
)

GPU:0
process       3393 uses     1126.000 MB GPU memory


Map:   0%|          | 0/61517 [00:00<?, ? examples/s]

In [26]:
## test DS filter
topic_embeddings_dataset.filter(lambda example: example["id"] in ['t_00004da3a1b2','t_0008768bdee6'])

Filter:   0%|          | 0/61517 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'description', 'channel', 'category', 'level', 'language', 'parent', 'has_content', 'id', 'concat_text', 'embeddings'],
    num_rows: 2
})

In [27]:
### dump topic dataset with embedding
data_save_path.mkdir(parents=True, exist_ok=True)
topic_embeddings_dataset.save_to_disk(data_save_path / 'topic_embeddings_dataset.hf')

Saving the dataset (0/1 shards):   0%|          | 0/61517 [00:00<?, ? examples/s]

In [29]:
if not iskaggle:
    push_notebook('saan', comp,
                  title='LECR: Sentence Transformer - Topic embedding',
                  file='/content/drive/MyDrive/Colab Notebooks/lecr_SentTrans_topic_emmbd.ipynb',
                  competition=comp, private=True, gpu=True)

Your kernel title does not resolve to the specified id. This may result in surprising behavior. We suggest making your title something that resolves to the specified id. See https://en.wikipedia.org/wiki/Clean_URL#Slug for more information on how slugs are determined.
Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/saansd2003/lecr-sentence-transformer-topic-embedding
